### S&P500 - Markowitz Optimization

In [179]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from urllib.request import urlopen
from gurobipy import *

In [172]:
#Using Yahoo Finance's historic prices csv API to query data of stocks in S&P500 index.
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'

#parameters are set to Nov. 2019 - Nov. 2020
params = {
    'period1': '1574656602',
    'period2': '1606279002',
    'interval': '1wk',
    'events':'history'
}

In [173]:
#Using BeautifulSoup to scrape stock tickers of companies in the S&P500 index from Wikipedia.
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser') 
table = soup.find('table',{'class':'wikitable sortable'}) #Locate the table on Wikipedia that contains ticker information
tickers = []
for row in table.findAll('tr')[1:]: #excluding the first row of the table that contains headers
    ticker = row.findAll('td')[0].text #extrating the first element (ticker symbol) in each data point
    sep = "\n"
    ticker = ticker.split(sep,1)[0] #parsing the extracted symbol since the scraped 'ticker' value contains '\n'
    tickers.append(ticker)
#cleaning: removing the following tickers because some have missing historical data (temporary delisted) on Yahoo Finance
for i in ['BRK.B','BF.B','CARR','LUMN','OTIS','VIAC','VNT','SWKS']:
    tickers.remove(i)
tickers.append('BRK-B') #adding back the ones that were format issues
tickers.append('BF-B')

In [177]:
#Creating price_data dataframe to contain price data points parsed from Yahoo Finance.
price_data = pd.DataFrame([])
for i in range(len(tickers)):
    response = requests.get(stock_url.format(tickers[i]),params = params) #tickers are from wikipedia, params are set for the period.
    file = StringIO(response.text)
    reader = csv.reader(file) #since the data downloaded is in csv format, very convinient for us, we will utilize reader to import the data points
    data = list(reader)[1:]
    price = []
    date = []
    for row in data:
        price.append(row[4]) #we only want closing prices of each week
        date.append(row[0]) #date of the weeks from first element
    price_data[tickers[i]] = price
price_data.index = date #setting the dataframe indexes to be the extracted dates

In [211]:
price_data.head(5)

,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,AES,...,XRX,XLNX,XYL,YUM,ZBRA,ZBH,ZION,ZTS,BRK-B,BF-B
2019-11-25,169.770004,85.449997,87.730003,196.179993,201.160004,54.830002,309.529999,39.150002,157.080002,18.910000,...,38.930000,92.779999,77.510002,100.669998,250.940002,145.279999,49.779999,120.519997,220.300003,67.820000
2019-12-02,171.470001,85.480003,86.980003,189.220001,202.550003,55.209999,306.230011,39.630001,154.619995,18.719999,...,37.770000,91.860001,75.320000,98.769997,256.920013,148.529999,49.709999,121.720001,222.610001,63.349998
2019-12-09,168.789993,86.349998,87.839996,181.850006,206.660004,58.650002,317.940002,41.150002,155.210007,18.920000,...,36.840000,96.230003,77.720001,100.110001,257.799988,148.070007,51.150002,124.449997,225.369995,63.849998
2019-12-16,175.369995,86.660004,89.290001,162.889999,211.100006,59.220001,327.609985,44.150002,158.119995,19.809999,...,37.349998,97.739998,78.610001,100.589996,252.490005,150.320007,51.630001,132.679993,226.309998,66.709999
2019-12-23,177.259995,87.400002,89.199997,169.270004,212.220001,59.189999,330.790009,46.180000,158.350006,19.980000,...,37.310001,98.419998,78.839996,101.900002,256.000000,150.009995,51.500000,133.250000,226.139999,67.940002


#### Markowitz Optimization Algorithm 

The idea is quite simple yet very powerful given the optimization toolkit we have. We would want to minimize the variance (volatility) of our portfolio subject to a return constraint. See below for the model's formulation.

\begin{align}
\underset{w}{\text{min}} \;\; & \sum_{i=1}^n \sum_{j=1}^n w_iw_jC_{ij}\\
\text{s.t.} \\
& \sum_{i=1}^n w_ir_i \ge \rho \\
& \sum_{i=1}^n w_i = 1 \\
& w_i \ge 0, \quad i = 1,2,\ldots,n \\
\end{align}

In [218]:
# get dimensions of data
d = price_data.shape[0]
n = price_data.shape[1] 

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
price = np.array(price_data)
price = price.astype(np.float)

for stock in range(n):
    for week in range(d-1):
        returns[week,stock] = price[week+1,stock]/price[week,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

In [257]:
#setting up the optimization model using Gurobi
mod = Model()

w_all = mod.addVars(n) #weights decision variables
x = mod.addVars(n, vtype = GRB.BINARY) #binary decision variables to determine inclusion/exclusion from our portfolio

#constraint for minimum monthly return of 0.5% 
mod.addConstr(sum(w_all[i] * avg_return[i] for i in range(n)) >= 0.005)

#constraint for adding that binary variable * weights of stocks to 1
mod.addConstr(sum(x[i] * w_all[i] for i in range(n)) == 1)
mod.addConstr(sum(w_all[i] for i in range(n)) == 1)

#constraint that weight must be greater than or equal to 0
for i in range(n):
    mod.addConstr(x[i] * w_all[i] >= 0)

#constraint that sum of binary variables must be less than or equal to 5
mod.addConstr(sum(x[i] for i in range(n)) <= 5)

#constraint that sum of binary variables must greater than 0
mod.addConstr(sum(x[i] for i in range(n)) >= 0)

#Objective Function
mod.setObjective(sum(sum(w_all[i] * w_all[j] * C[i][j] for i in range(n))for j in range(n)), GRB.MINIMIZE)

In [258]:
mod.Params.TimeLimit = 300.0 #limiting the model computation time to 5 mins
mod.update()
mod.optimize()

Changed value of parameter TimeLimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4 rows, 998 columns and 1996 nonzeros
Model fingerprint: 0xaec52f73
Model has 124750 quadratic objective terms
Model has 500 quadratic constraints
Variable types: 499 continuous, 499 integer (499 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-03, 5e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.06s
Presolved: 1501 rows, 1497 columns, 5489 nonzeros
Presolved model has 124750 quadratic objective terms
Variable types: 998 continuous, 499 integer (499 binary)
Found heuristic solution: objective 0.0024568

Root relaxation: objec

In [259]:
a = list(x) #x is the gurobi object that represents our decision variables, getting the indexes from the object.
b = []
c = []
for i in range(len(x)):
    b.append(x[i].X) #appending the model's decision of either 1 or 0 to a specific stock.
for i in range(len(x)):
    c.append(w_all[i].X) #appending the weights of the stocks that are assigned with '1' for their decision variable x.
df = pd.DataFrame(list(zip(a,b,c))) #zipping the three components (index,decision variable, weights)
df2 = df[df[1]==1] #subsetting a new dataframe that contains the 5 stocks that want
comb = list(df[df[1]==1].index)
portfolio = []
for i in comb:
    portfolio.append(tickers[i]) #getting the tickers of the stocks
opt_val = mod.objval
print(f'Optimal Risk Value: {opt_val}\n')
print(f'Model Solve Time: {mod.runtime:.2f} seconds\n')
print('Weights of the portfolio with lowest volatility:')
for i in range(5):
    print(f'{portfolio[i]}: {df2.iloc[i,2] * 100:.4f}%')

Optimal Risk Value: 0.000352894004471579

Model Solve Time: 2.09 seconds

Weights of the portfolio with lowest volatility:
CLX: 24.7285%
KR: 17.1464%
PWR: 8.5392%
NOW: 13.3857%
TIF: 36.2002%
